In [ ]:
import csv
import json
import re
import time
from urllib.parse import urljoin

import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

df = pd.read_csv(r'C:\Users\Dabu bahya\Downloads\Desktop\Review-Analysis\new_product_list.csv')


new_df = df.query('`Category`.str.startswith("data mining")')



# We start scraping each product site with time delay of 10s.
start_time = time.time()

for i in range(3,4):
    url =  "https://" + new_df.iloc[i]['URL to Capterra Page']
    p_name = new_df.iloc[i]['Product Name']
    p_category = new_df.iloc[i]['Category']
    #     print(p_name)  to be removed
    #   print(p_category) to be removed

    if re.match(r"#reviews", url):
        url = re.sub(r"#reviews",'', url)
    else:
        pass
    options = webdriver.ChromeOptions()
    options.headless = True
    driver = webdriver.Chrome(r'D:\Downloads\Desktop\chromedriver',chrome_options=options)  
    driver.get(url)

    soup = BeautifulSoup(driver.page_source,"html.parser" )
    soup_obj = soup.find('div',{'class':'ProductSummary__SummaryPanel-mcm4i7-6 hCwGXf'})

    product = soup_obj.contents[0]
    product_info = product.select('div > div')[0].get_text(strip=True) #this is info about the product 
    #   print(product_info) to be removed
    #for checking that if "featured in" exists on the site 
    x = soup_obj.contents[1].h2.get_text()

    if re.match(r"Featured In", x):

        c = soup_obj.contents[2]
        y = c.h2.get_text(strip=True)
        if re.match(r"Best For",y):
            d = c.select('div > em')[0].get_text(strip=True) # best for 
            var = 3

        else:
            d = "Not Available"
            var = 2


        try :
            e = soup_obj.contents[var]
            f = e.select('div > p')[0].get_text(strip=True)  # company name
        except:
            f = "Not Available"

        try :
            g = soup_obj.contents[var]
            h = g.select('div > p')[1].get_text(strip=True)  # product website
        except:
            h = "Not Available"  


        i = soup_obj.contents[var]
        j = i.select('div > p')[2].get_text(strip=True)  # founding year
        if not j:
            j = "Not Available"
        else:
            pass  

        k = soup_obj.contents[var]
        l = k.select('div > p')[3].get_text(strip=True)  # founding place
        if not l:
            l = "Not Available"
        else:
            pass

    else:
        c = soup_obj.contents[1]
        y = c.h2.get_text(strip=True)
        if re.match(r"Best For",y):
            d = c.select('div > em')[0].get_text(strip=True)
            var = 2

        else:
            d = "Not Available"
            var = 1


        try :
            e = soup_obj.contents[var]
            f = e.select('div > p')[0].get_text(strip=True)
        except:
            f = "Not Available"

        try :
            g = soup_obj.contents[var]
            h = g.select('div > p')[1].get_text(strip=True)
        except:
            h = "Not Available"

        i = soup_obj.contents[var]
        j = i.select('div > p')[2].get_text(strip=True)
        if not j:
            j = "Not Available"
        else:
            pass

        k = soup_obj.contents[var]
        l = k.select('div > p')[3].get_text(strip=True)
        if not l:
            l = "Not Available"
        else:
            pass
    info = {'About Product': product_info,'Best For':d,'Company Name': f,'Product Website': h, 'Founding Year': j,'Founding Place': l }


    new_soup = soup.find('div',{'class':'SpecTable__Root-sc-1jq5uy4-0 jZHgUG'})
    len_of_new_soup = len(new_soup.contents)
    # the listed price of the product, if given.
    try:
        price_heading = new_soup.contents[0].h3.get_text()
        price = new_soup.contents[0].span.get_text()
        listed_price = {price_heading:price}

    except:
        pass
    try:
        price_heading = new_soup.contents[0].h3.get_text()
        price = new_soup.contents[0].a.get_text() #there's a link attached with the price 
        listed_price = {price_heading:price}
    except:
        pass
    try:
        price_heading = new_soup.contents[0].h3.get_text()
        price = new_soup.contents[0].select('li > div').get_text()
        listed_price = {price_heading:price}
    except:
        pass
    # for available features in Deployment, Training,& Support
    ndic ={}
    for i in range(1,len_of_new_soup):
        s_obj = new_soup.contents[i]
        heading = s_obj.h3.get_text()
        new_obj = s_obj.contents[1]
        nl = []
        for i in range(len(new_obj)):
            an_obj = new_obj.contents[i]
            x = an_obj.select('div > div')[0]['class']
            y= x[1]
            if y == 'bqMKDC':
                q = an_obj.select('div > div')[1].get_text(strip=True)
                nl.append(q)
            if not nl:
                pass
            else:
                ndic[heading] =nl
        
    #  print(ndic)  to be removed


    another_soup_obj = soup.find('div',{'class':'Accordion__Root-sc-1jo79it-0 jGWhUg'})
    # clicking all the buttons 
    for i in range(1,len(another_soup_obj.contents)):
        h4 = another_soup_obj.contents[i].h4.get_text()
        cb = "//button[normalize-space()=" + f'"{h4}"' + "]"
        element = driver.find_element_by_xpath(cb)
        driver.execute_script("arguments[0].click();", element)

    # extracting the corresponding features
    nd = {}
    for i in range(len(another_soup_obj)):
        field_heading = another_soup_obj.contents[i].h4.get_text()
        n_o_j = another_soup_obj.contents[i]
        xyz = n_o_j.find_all('div',{'class':'FeatureList__FeatureCell-sc-1kxeq27-1 ifqTCQ'})
        nl1 = []
        for i in xyz:
            zas = i.select('div > div')[1]['class']
            sac = zas[1]
            if sac == 'bqMKDC':
                q = i.select('div > div')[2].get_text(strip=True)
                nl1.append(q)
            if  not nl1:
                    pass
            else:
                nd[field_heading] = nl1
        
    print(nd)  #to be removed
    driver.close()

    newurl = urljoin(url,'reviews')
    another_options = webdriver.ChromeOptions()
    another_options.headless = True
    another_driver = webdriver.Chrome(r'D:\Downloads\Desktop\chromedriver',chrome_options=another_options)
    another_driver.get(newurl)

    while True:
        try:
            #another_driver.execute_script("arguments[0].scrollIntoView(true);", WebDriverWait(another_driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//button[normalize-space()="Show more reviews"]'))))
            #another_driver.execute_script("arguments[0].click();", WebDriverWait(another_driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[normalize-space()="Show more reviews"]'))))
            new_element = another_driver.find_element_by_xpath('//button[normalize-space()="Show more reviews"]')
            another_driver.execute_script("arguments[0].click();", new_element)
        
        except:
            break

    review_soup = BeautifulSoup(another_driver.page_source,"html.parser" )

    ls = ['Overall','Ease of Use','Customer Service']
    ratings = soup.find_all('div',{'class':'StarRating__Rating-sc-9jwzgg-1 cAGyvf'})
    la = []
    for x,y in zip(ls,ratings):
        la.append({x:y.get_text(strip=True)})
    print(la)
    start = review_soup.find('div',{'class':'gtm-review-section'})
    #print((start.contents))
    try:
        
        perm_list = []

        for i in range(len(start.contents)-1):
            empty_list = []
        #reviewer info
            qw = start.contents[i]
            personal_info = qw.find('div',{'class':'ReviewerAvatarSection__ReviewerInfo-sc-1a80501-1 cSadMy'})
            name = personal_info.contents[0].get_text(strip=True)
            other_info = personal_info.contents[1]
            designation = other_info.select('div > div')[0].get_text(strip=True)
            working_domain = other_info.select('div > div')[1].get_text(strip=True) 
            usage = other_info.select('div > div')[2].get_text(strip=True)
            reviewer_info = {name:[designation,working_domain,usage]}
            empty_list.append(reviewer_info)

        #reviewer ratings    
            li = ['Overall','Ease of Use','Customer Service', 'Features', ' Value for Money','Likelihood to Recommend']
            ra = qw.find_all('div',{'class':'StarRating__Rating-sc-9jwzgg-1 kCLiav'})
            reviewer_ratings = []
            for x,y in zip(li,ra):
                reviewer_ratings.append({x:y.get_text(strip=True)})
            empty_list.append(reviewer_ratings)

        #date    
            date = qw.find('div',{'class':'ReviewSource__Root-lnjke6-0 iBAGLQ'}).contents[2].get_text(strip=True)
            empty_list.append(date)

        #review

            review = qw.find('div',{'class':'ReviewCard__RightFlexItem-sc-18j15p9-3 cOmWnW ReviewCard__FlexItem-sc-18j15p9-2 bTVfYT'})
            review_heading = review.contents[0].h3.get_text(strip=True)
            overall = review.contents[0].find_all('div',{'class':'ReviewSection__Root-sc-189472c-0 icjcMH'})
            review_text ={}
            review_text['review heading'] = review_heading
            for i in overall :
                text = i.get_text(strip=True)
                manip = text.split(':')
                header = manip[0]
                review_content = manip[1]
                review_text[header] = review_content


            empty_list.append(review_text)
            perm_list.append(empty_list)
        #print(perm_list)
        new_dic = {'Product Name': p_name,'Category': p_category, 'Product Info': info , 'Price': listed_price, 'Overall Features': ndic ,'Category Features': nd ,'Overall Ratings': la, 'Review': perm_list}
        #print(new_dic)
        other = pd.DataFrame([new_dic])
        #print(other)
        with open('another_spss.csv', 'a', encoding='utf8') as f:
            other.to_csv(f, header=f.tell()==0)
    except:
        pass
    another_driver.close()   
    time.sleep(10)

end_time=time.time()
total_time=end_time-start_time   


print(total_time)
print(len(perm_list))





<ipython-input-3-9521ee252c8c>:38: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(r'D:\Downloads\Desktop\chromedriver',chrome_options=options)


{'Data Extraction': ['Disparate Data Collection', 'Email Address Extraction', 'IP Address Extraction', 'Image Extraction', 'Phone Number Extraction', 'Pricing Extraction', 'Web Data Extraction']}


<ipython-input-3-9521ee252c8c>:206: DeprecationWarning: use options instead of chrome_options
  another_driver = webdriver.Chrome(r'D:\Downloads\Desktop\chromedriver',chrome_options=another_options)
